In [ ]:
# Vaccination Second Dose Uptake By Vaccination Date Percentage stats for Lewisham
# Loads the data and prepares it for use in Lewisham ward profiles
#
# MSOAs data, mapped to wards

In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
import google.colab.files as files

import IPython

Data
==
GDrive mount
--

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Used as root folder.
project_dir = '/content/gdrive/MyDrive/WardProfiles'
ward_lookups_dir = f"{project_dir}/lookups/2022_wards"

# For exports
out_dir = f"{project_dir}/covid_vaccination_uptake"

In [4]:
!mkdir -p '{out_dir}'

Downloads
--

In [5]:
# 20-12-2021, Vaccination Second Dose Uptake By Vaccination Date Percentage MSOA level
# Source: 
# https://coronavirus.data.gov.uk/
!wget 'https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E09000023&metric=cumVaccinationSecondDoseUptakeByVaccinationDatePercentage&format=csv' \
  -O "{out_dir}/cumVaccinationSecondDoseUptakeByVaccinationDatePercentage.csv"

--2021-12-20 18:00:43--  https://api.coronavirus.data.gov.uk/v2/data?areaType=msoa&areaCode=E09000023&metric=cumVaccinationSecondDoseUptakeByVaccinationDatePercentage&format=csv
Resolving api.coronavirus.data.gov.uk (api.coronavirus.data.gov.uk)... 13.107.246.69, 13.107.213.69, 2620:1ec:bdf::69, ...
Connecting to api.coronavirus.data.gov.uk (api.coronavirus.data.gov.uk)|13.107.246.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3834 (3.7K) [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/covid_vaccination_uptake/cumVaccinationSecondDoseUptakeByVaccinationDatePercentage.csv’

/content/gdrive/MyD 100%[===================>]   3.74K  --.-KB/s    in 0s      

2021-12-20 18:00:44 (325 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/covid_vaccination_uptake/cumVaccinationSecondDoseUptakeByVaccinationDatePercentage.csv’ saved [3834/3834]



In [6]:
!ls -lh "{out_dir}"

total 11K
-rw------- 1 root root 3.8K Dec 20 17:56 cumVaccinationFirstDoseUptakeByVaccinationDatePercentage.csv
-rw------- 1 root root 3.8K Dec 20 18:00 cumVaccinationSecondDoseUptakeByVaccinationDatePercentage.csv
-rw------- 1 root root 2.6K Dec 20 17:58 lbl_vacc_uptake_1st_pc_msoa11.csv


Lookups
--
Used to match up MSOAs with 2022 electoral wards

In [7]:
# All MSOAs and their 2022 wards in Lewisham
msoa_wd22_join = pd.read_csv(f"{ward_lookups_dir}/lbl_msoa11_wd22_proposed.csv")
msoa_wd22_join = msoa_wd22_join[['MSOA11CD', 'WD22CD_proposed']]
msoa_wd22_join.head()

,MSOA11CD,WD22CD_proposed
0,E02006784,E05013720
1,E02006783,E05013720
2,E02000666,E05013725
3,E02000663,E05013725
4,E02000667,E05013718


Process & export
==

In [8]:
# For filtering of source data
lbl_msoa_list = msoa_wd22_join.MSOA11CD.unique() # All MSOAs in Lewisham
len(lbl_msoa_list)

36

In [9]:
# Source data
df = pd.read_csv(f"{out_dir}/cumVaccinationSecondDoseUptakeByVaccinationDatePercentage.csv") 
df.head()

,regionCode,regionName,UtlaCode,UtlaName,LtlaCode,LtlaName,areaCode,areaName,areaType,date,cumVaccinationSecondDoseUptakeByVaccinationDatePercentage
0,E12000007,London,E09000023,Lewisham,E09000023,Lewisham,E02000653,Silwood & Pepys,msoa,2021-12-19,58.0
1,E12000007,London,E09000023,Lewisham,E09000023,Lewisham,E02000654,Deptford North,msoa,2021-12-19,54.5
2,E12000007,London,E09000023,Lewisham,E09000023,Lewisham,E02000655,New Cross Gate,msoa,2021-12-19,55.6
3,E12000007,London,E09000023,Lewisham,E09000023,Lewisham,E02000657,New Cross South,msoa,2021-12-19,56.7
4,E12000007,London,E09000023,Lewisham,E09000023,Lewisham,E02000658,Telegraph Hill,msoa,2021-12-19,61.1


In [10]:
# Standardise column names
df = df.rename(columns={
    'areaCode': 'MSOA11CD', 
    'areaName': 'MSOA11NM',
    'LtlaCode': 'LAD19CD',
    'LtlaName': 'LAD19NM',
    'cumVaccinationSecondDoseUptakeByVaccinationDatePercentage': 'vacc_uptake_2nd_pc'
                      })
df.columns

Index(['regionCode', 'regionName', 'UtlaCode', 'UtlaName', 'LAD19CD',
       'LAD19NM', 'MSOA11CD', 'MSOA11NM', 'areaType', 'date',
       'vacc_uptake_2nd_pc'],
      dtype='object')

In [11]:
df1 = df[['MSOA11CD', 'MSOA11NM', 'LAD19CD', 'LAD19NM', 'date', 'vacc_uptake_2nd_pc']]

In [12]:
df1.head()

,MSOA11CD,MSOA11NM,LAD19CD,LAD19NM,date,vacc_uptake_2nd_pc
0,E02000653,Silwood & Pepys,E09000023,Lewisham,2021-12-19,58.0
1,E02000654,Deptford North,E09000023,Lewisham,2021-12-19,54.5
2,E02000655,New Cross Gate,E09000023,Lewisham,2021-12-19,55.6
3,E02000657,New Cross South,E09000023,Lewisham,2021-12-19,56.7
4,E02000658,Telegraph Hill,E09000023,Lewisham,2021-12-19,61.1


In [13]:
df1.shape

(36, 6)

In [15]:
# sanity check for Lewisham MSOAs
lbl_vacc_uptake = df1[df1.MSOA11CD.isin(lbl_msoa_list)].copy()

In [16]:
lbl_vacc_uptake

,MSOA11CD,MSOA11NM,LAD19CD,LAD19NM,date,vacc_uptake_2nd_pc
0,E02000653,Silwood & Pepys,E09000023,Lewisham,2021-12-19,58.0
1,E02000654,Deptford North,E09000023,Lewisham,2021-12-19,54.5
2,E02000655,New Cross Gate,E09000023,Lewisham,2021-12-19,55.6
3,E02000657,New Cross South,E09000023,Lewisham,2021-12-19,56.7
4,E02000658,Telegraph Hill,E09000023,Lewisham,2021-12-19,61.1
5,E02000659,Blackheath West,E09000023,Lewisham,2021-12-19,61.3
6,E02000660,Brockley East & St Johns,E09000023,Lewisham,2021-12-19,61.4
7,E02000661,Blackheath Village,E09000023,Lewisham,2021-12-19,67.5
8,E02000662,Brockley West,E09000023,Lewisham,2021-12-19,56.0
9,E02000663,Loampit & Hilly Fields,E09000023,Lewisham,2021-12-19,63.6


In [17]:
# add ward codes
msoa_wd22_map = msoa_wd22_join.set_index('MSOA11CD').WD22CD_proposed.to_dict()
lbl_vacc_uptake['WD22CD_proposed'] = lbl_vacc_uptake.MSOA11CD.map(lambda msoa: msoa_wd22_map[msoa])

In [18]:
# Export
lbl_vacc_uptake.to_csv(f"{out_dir}/lbl_vacc_uptake_2nd_pc_msoa11.csv", index=False)
IPython.display.display(len(lbl_vacc_uptake))
IPython.display.display(lbl_vacc_uptake.head())

36

,MSOA11CD,MSOA11NM,LAD19CD,LAD19NM,date,vacc_uptake_2nd_pc,WD22CD_proposed
0,E02000653,Silwood & Pepys,E09000023,Lewisham,2021-12-19,58.0,E05013721
1,E02000654,Deptford North,E09000023,Lewisham,2021-12-19,54.5,E05013719
2,E02000655,New Cross Gate,E09000023,Lewisham,2021-12-19,55.6,E05013728
3,E02000657,New Cross South,E09000023,Lewisham,2021-12-19,56.7,E05013716
4,E02000658,Telegraph Hill,E09000023,Lewisham,2021-12-19,61.1,E05013732
